# Authoring Leaf Systems
notebook 실행 방법은 [index](./index.ipynb)를 참조하자.


In [2]:
import numpy as np
from pydrake.common.containers import namedview
from pydrake.common.value import Value
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import BasicVector, LeafSystem
from pydrake.trajectories import PiecewisePolynomial

## Overview

[Modeling Dynamical Systems](./dynamical_systems.ipynb) 튜터리얼에서 기본 [LeafSystem](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_leaf_system.html)을 작성해보면서 Drake 시스템 프레임워크에 대한 기본적인 지식을 얻도록 한다. 이 notebook에서 좀더 고급/완전한 시스템을 만드는데 대한 개요를 제공한다.

## Input and Output Ports

Leaf 시스템은 임의의 개수의 입력 및 출력 포트를 가질 수 있다. 포트에는 두 가지 기본 유형이 있습니다: 벡터 값(**vector-valued**) 포트와 추상 값(**abstract-valued**) 포트이다.

### Vector-valued Ports
벡터 값 포트부터 시작해보자. 이 포트는 사용하기가 가장 간단하다. 다음 시스템은 두 개의 벡터 입력 포트와 두 개의 벡터 출력 포트를 선언한다. 이 시스템은 두 개의 2-element 입력 벡터의 합과 차를 계산하여 결과를 두 개의 출력 포트에 놓습니다.

<table align=center cellpadding=0 cellspacing=0><tr align=center style="border:none;"><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=right style="padding:5px 0px 5px 0px; border:none;">a &rarr;</td></tr><tr style="border:none;"><td align=right style="padding:5px 0px 5px 0px; border:none;">b &rarr;</td></tr></table></td><td align=center style="border:2px solid black;padding-left:20px;padding-right:20px;vertical-align:middle;" bgcolor=#F0F0F0>MyAdder</td><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; sum</td></tr><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; difference</td></tr></table></td></tr></table>

In [ ]:
class MyAdder(LeafSystem):
    def __init__(self):
        super().__init__()  # 반드시 base class를 초기화하자!
        self._a_port = self.DeclareVectorInputPort(name="a", size=2)
        self._b_port = self.DeclareVectorInputPort(name="b", size=2)
        self.DeclareVectorOutputPort(name="sum", size=2, calc=self.CalcSum)
        self.DeclareVectorOutputPort(name="difference",
                                     size=2,
                                     calc=self.CalcDifference)

    def CalcSum(self, context, output):
        # 2x1 벡터를 얻기 위해서 input ports를 evaluation한다.
        a = self._a_port.Eval(context)
        b = self._b_port.Eval(context)

        # 합을 출력 벡터에 쓴다.
        output.SetFromVector(a + b)

    def CalcDifference(self, context, output):
        # 2x1 벡터를 얻기 위해서 input ports를 evaluation한다.
        a = self._a_port.Eval(context)
        b = self._b_port.Eval(context)

        # 차를 출력 벡터에 쓴다.
        output.SetFromVector(a - b)

# 이 system의 instance와 context를 생성한다.
system = MyAdder()
context = system.CreateDefaultContext()

# 입력 포트를 고정 값으로 고정한다.
system.GetInputPort("a").FixValue(context, [3, 4])
system.GetInputPort("b").FixValue(context, [1, 2])

# 출력 포트를 evaluation한다.
print(f"sum: {system.GetOutputPort('sum').Eval(context)}")
print(f"difference: {system.GetOutputPort('difference').Eval(context)}")

몇 가지 명심해야할 사항들:
- 원하는 만큼의 입력 또는 출력을 선언할 수 있다(크기가 동일할 필요는 없음).
- Context를 허용하는 시스템 메서드 중 하나에서 Eval() 메서드를 통해 입력을 평가할 수 있다.
- 각 출력 포트에 대해 출력을 구현하는 다른 콜백 함수를 정의한다.
- 포트의 순서를 추적하는 것은 다소 번거로울 수 있다. 따라서 인덱스를 사용하거나 이름으로 참조하는 등 코드를 개선할 수 있다.

다음을 이해하고 있으면 도움이 된다:
- 시스템 메소드 구현에서 [InputPort::HasValue()](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_input_port.html#a5536b94a4642fa4cf47164437dc66ae8)를 확인하여 입력을 선택 사항으로 만들 수도 있다.
- 입력 및 출력 포트는 자체적으로 타이밍 의미를 가지지 않는다. 출력 포트는 출력이 요청될 때마다 평가되며, 반복 계산을 피하기 위해 [캐싱](https://drake.mit.edu/doxygen_cxx/group__cache__design__notes.html)을 기본적으로 사용한다.
- `DeclareVectorOutputPort()` 및 `DeclareAbstractOutputPort()` 포트는 캐싱 시스템에서 출력 포트를 다시 evaluation해야 하는 시기를 결정하는 데 사용되는 옵션 인수 prerequisites_of_calc를 사용합니다. 좁은 전제 조건을 선언하면 코드 속도를 높일 수 있다. 놀랍게도, 전제 조건을 명시적으로 좁히면 일부 Diagram의 생성 속도를 크게 높일 수도 있다. 왜냐하면 이러한 사항이 지정되지 않은 경우 DiagramBuilder는 시스템을 기호 형식으로 변환하여 대수 루프를 확인하려고 시도하기 때문이다.
- The `DeclareVectorOutputPort()` and `DeclareAbstractOutputPort()` ports take an optional `prerequisites_of_calc` argument which is used by the caching system to determine when the output port needs to be re-evaluated. Declaring narrow prerequisites can speed up your code. Surprisingly, explicitly narrowing the prerequisites can also dramatically speed up the construction of some `Diagram`s, because when these are not specified the `DiagramBuilder` attempts to check for algebraic loops by converting your systems into symbolic form.


### Abstract-valued Ports
모든 입력과 출력을 벡터로 표현하는 것이 가장 좋은 방법은 아니다. Drake 시스템 프레임워크에서는 구조화된 타입(structured types)을 포트를 통해 전달하는 기능도 제공하는데, 이는 C++에서 "타입 소거(type erasure)"라는 기술을 사용하여 구현된다. 시스템 프레임워크의 관점에서 볼 때, 클래스들은 "추상적인" 데이터 타입을 다루며, 시스템 구현 자체만이 구조화된 타입을 다루는 방법을 알면 된다. 실제로 약간의 보일러플레이트(반복적인 코드)만 무시할 수 있다면 사용법은 간단하다. 입력 및 출력 포트에서 `RigidTransform`을 사용하는 `LeafSystem`의 예를 살펴보자. :

<table align=center cellpadding=0 cellspacing=0><tr align=center><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=right style="padding:5px 0px 5px 0px; border:none;">in &rarr;</td></tr></table></td><td align=center style="border:2px solid black;padding-left:20px;padding-right:20px;vertical-align:middle" bgcolor=#F0F0F0>RotateAboutZ</td><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; out</td></tr></table></td></tr></table>

In [ ]:
class RotateAboutZ(LeafSystem):
    def __init__(self):
        super().__init__()  # base class를 반드시 초기화!
        self.DeclareAbstractInputPort(name="in",
                                      model_value=Value(RigidTransform()))
        self.DeclareAbstractOutputPort(
            name="out",
            alloc=lambda: Value(RigidTransform()),
            calc=self.CalcOutput)

    def CalcOutput(self, context, output):
        # RigidTransform를 얻기 위해 입력 포트를 evaluation하기
        X_1 = system.get_input_port().Eval(context)

        X_2 = RigidTransform(RotationMatrix.MakeZRotation(np.pi / 2)) @ X_1

        # 출력 RigidTransform을 설정
        output.set_value(X_2)

# 이 시스템의 인스턴스와 context를 생성
system = RotateAboutZ()
context = system.CreateDefaultContext()

# 입력 포트를 상수값으로 고정
system.get_input_port().FixValue(context, RigidTransform())

# 출력 포트를 evaluation한다.
print(f"output: {system.get_output_port(0).Eval(context)}")

## State Variables

시스템은 `Context`에 상태(state)를 저장하며, 입출력 포트와 매우 비슷한 방식으로 상태를 처리합니다. 벡터 값 상태(vector-valued) 또는 추상 값(abstract-valued) 상태를 사용할 수 있다. 추상 상태는 이산 시간(discrete-time) 방식으로만 업데이트할 수 있는 반면, 벡터 값 상태는 이산(**discrete**) 또는 연속(**continuous**)으로 선언될 수 있다.  

이산 상태 $x_d$ 는 업데이트 이벤트에 따라 진화(evolve)하며, 가장 일반적인 형태는 **차분 방정식(difference equation)**을 정의하는 단순한 주기적 이벤트이다. [notebook 소개](./dynamical_systems.ipynb)에서 이에 대한 예제를 보았었다.

In [ ]:
class SimpleDiscreteTimeSystem(LeafSystem):
    def __init__(self):
        super().__init__()

        state_index = self.DeclareDiscreteState(1)  # One state variable.
        self.DeclareStateOutputPort("y", state_index)  # One output: y=x.
        self.DeclarePeriodicDiscreteUpdateEvent(
            period_sec=1.0,  # 1초 마다 step
            offset_sec=0.0,  # 첫번째 event는 time 0에서
            update=self.Update) # 아래 정의된 Update method를 호출

    # x[n+1] = x^3[n].
    def Update(self, context, discrete_state):
        x = context.get_discrete_state_vector().GetAtIndex(0)
        x_next = x**3
        discrete_state.get_mutable_vector().SetAtIndex(0, x_next)

# 시스템 인스턴스
system = SimpleDiscreteTimeSystem()
simulator = Simulator(system)
context = simulator.get_mutable_context()

# 초기 조건 설정 : x[0] = [0.9].
context.get_mutable_discrete_state_vector().SetFromVector([0.9])

# simulation 실행
simulator.AdvanceTo(4.0)
print(context.get_discrete_state_vector())

이러한 이벤트의 구현 및 순서에 대한 자세한 내용은 [여기](https://drake.mit.edu/doxygen_cxx/group__discrete__systems.html)에서 확인할 수 있다. 이산 변수(discrete variables)를 업데이트하기 위해 더 복잡한 [이벤트](https://drake.mit.edu/doxygen_cxx/group__events__description.html)를 정의하는 것도 가능하다.

`다이어그램` 내에서 다른 시스템들에 한 시스템의 상태에 직접 액세스할 수 없다는 점에 유의하시오. 시스템이 상태를 공유하려면 출력 포트를 사용해야 한다. 이러한 일반적인 경우를 쉽게 처리할 수 있도록 `DeclareStateOutputPort()` 메서드를 제공한다.

연속 벡터 값 상태 변수(continuous vector-valued state variables)의 진화는 미분 방정식(differential equation)에 의해 제어된다. 여러 개의 서로 다른 이산 상태 변수(discrete state variables) 그룹을 선언하는 것은 가능하고 편리하지만(심지어 서로 다른 이벤트에 의해 서로 다른 비율(속도)로 업데이트될 수 있음), 시스템에 대해 0개 또는 1개의 연속 상태 벡터(continuous state vector)만 정의하고 `LeafSystem::DoCalcTimeDerivatives()` 메서드를 오버로드하여 동역학을 정의한다. [notebook 소개](./dynamical_systems.ipynb)에서 간단한 연속 시간(continuous-time)에 대한 예제는 아래와 같다. :

In [ ]:
# 시스템 정의
class SimpleContinuousTimeSystem(LeafSystem):
    def __init__(self):
        super().__init__()

        state_index = self.DeclareContinuousState(1)  # 상태변수 한개
        self.DeclareStateOutputPort("y", state_index)  # 출력 한개. One output: y=x.

    # xdot(t) = -x(t) + x^3(t).
    def DoCalcTimeDerivatives(self, context, derivatives):
        x = context.get_continuous_state_vector().GetAtIndex(0)
        xdot = -x + x**3
        derivatives.get_mutable_vector().SetAtIndex(0, xdot)

# 시스템 인스턴스
system = SimpleContinuousTimeSystem()
simulator = Simulator(system)
context = simulator.get_mutable_context()

# 초기화 조건 설정: x(0) = [0.9]
context.SetContinuousState([0.9])

# simulation 실행
simulator.AdvanceTo(4.0)
print(context.get_continuous_state_vector())

추상 값 상태(abstract-valued state)의 경우 유사한 워크플로우를 따른다. 생성자에서 추상 상태를 선언하고 해당 상태를 업데이트하기 위한 업데이트 이벤트를 정의한다. 이산 업데이트 이벤트(discrete update)는 이산 상태(discrete states)만 업데이트하도록 제한되므로 추상 상태를 업데이트하기 위해 "제한 없는(unrestricted)" 이벤트를 사용한다. 다음은 간단한 `PiecewisePolynomial` 궤적을 추상 상태로 저장하는 예이다. :

In [ ]:
class AbstractStateSystem(LeafSystem):
    def __init__(self):
        super().__init__()

        self._traj_index = self.DeclareAbstractState(
            Value(PiecewisePolynomial()))
        self.DeclarePeriodicUnrestrictedUpdateEvent(period_sec=1.0,
                                                    offset_sec=0.0,
                                                    update=self.Update)

    def Update(self, context, state):
        t = context.get_time()
        traj = PiecewisePolynomial.FirstOrderHold(
            [t, t + 1],
            np.array([[-np.pi / 2.0 + 1., -np.pi / 2.0 - 1.], [-2., 2.]]))
        # state를 업데이트
        state.get_mutable_abstract_state(int(self._traj_index)).set_value(traj)



system = AbstractStateSystem()
simulator = Simulator(system)
context = simulator.get_mutable_context()

# 추상 상태(abstract state)를 위한 초기 조건을 설정
context.SetAbstractState(0, PiecewisePolynomial())

# simulation 실행
simulator.AdvanceTo(4.0)
traj = context.get_abstract_state(0).get_value()
print(f"breaks: {traj.get_segment_times()}")
print(f"traj({context.get_time()}) = {traj.value(context.get_time())}")

## Parameters

매개변수는 상태와 유사하지만 시뮬레이션의 수명 동안 일정하다는 점이 다르다. 시스템 프레임워크내에서 매개변수는 상태(state)와 거의 동일하게 선언되고 액세스되지만 업데이트되지 않는다. 여기에서도 벡터 값(`DeclareNumericParameter`로 선언) 및 추상 값(`DeclareAbstractParameter`를 통해) 매개변수를 사용할 수 있다.

<table align=center cellpadding=0 cellspacing=0><tr align=center><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0></table></td><td align=center style="border:2px solid black;padding-left:20px;padding-right:20px;vertical-align:middle" bgcolor=#F0F0F0>SystemWithParameters</td><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; numeric</td></tr><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; abstract</td></tr></table></td></tr></table>

In [ ]:
class SystemWithParameters(LeafSystem):
    def __init__(self):
        super().__init__()  # base class 초기화해야만 한다!

        self.DeclareNumericParameter(BasicVector([1.2, 3.4]))
        self.DeclareAbstractParameter(
            Value(RigidTransform(RotationMatrix.MakeXRotation(np.pi / 6))))

        # 출력 포트를 선언하여 system methods내에서 parameters에 접근하는 방법을 보여준다.
        self.DeclareVectorOutputPort(name="numeric",
                                     size=2,
                                     calc=self.OutputNumeric)
        self.DeclareAbstractOutputPort(
            name="abstract",
            alloc=lambda: Value(RigidTransform()),
            calc=self.OutputAbstract)

    def OutputNumeric(self, context, output):
        output.SetFromVector(context.get_numeric_parameter(0).get_value())

    def OutputAbstract(self, context, output):
        output.set_value(context.get_abstract_parameter(0).get_value())

# 시스템의 인스턴스와 context 생성
system = SystemWithParameters()
context = system.CreateDefaultContext()

# 출력 포트를 evaluation한다.
print(f"numeric: {system.get_output_port(0).Eval(context)}")
print(f"abstract: {system.get_output_port(1).Eval(context)}")

## 시스템은 publish가 가능(Systems can "publish")

상태를 업데이트하는 방법과 출력 포트를 평가(evaluation)하는 방법 외에도 LeafSystem에서 지원하는 또 다른 방법은 "publish"에 대한 콜백입니다. "publish" 메서드는 상태를 수정할 수 없다. : 시스템 프레임워크 외부로 데이터를 브로드캐스트(broadcasting)하고(예: ROS에서의 메시지 전달 프로토콜을 통해), 시뮬레이션을 종료하고, 오류를 감지하고, 통합 단계 간에 경계를 강제하는 데 유용하다. 이 메서드는 다른 시스템 콜백과 매우 유사한 방식으로 선언된다.

In [ ]:
class MyPublishingSystem(LeafSystem):
    def __init__(self):
        super().__init__()

        # `ForcePublish()` 호출은 해당 callback이 트리거된다. 
        self.DeclareForcedPublishEvent(self.Publish)

        # 1초에 한번씩 publish
        self.DeclarePeriodicPublishEvent(period_sec=1,
                                         offset_sec=0,
                                         publish=self.Publish)
        
    def Publish(self, context):
        print(f"Publish() called at time={context.get_time()}")

system = MyPublishingSystem()
simulator = Simulator(system)
simulator.AdvanceTo(5.3)

# 임의 시간에 publish를 "강제"할 수도 있다.
print("\ncalling ForcedPublish:")
system.ForcedPublish(simulator.get_context())


## Naming Vector Values

추상 값 포트(abstract-valued ports), 상태(state) 및/또는 매개변수(parameters)는 구조화된 데이터로 사용할 수 있다. 그러나 벡터 값 데이터(vector-valued data)의 개별 요소를 참조하기 위해 문자열 이름을 사용하는 것이 편리하다. Python에서는 Python의 [`namedtuple`](https://docs.python.org/3.6/library/collections.html?highlight=namedtuple#collections.namedtuple)에서 영감을 받은 [`namedview`](https://drake.mit.edu/pydrake/pydrake.common.containers.html?highlight=namedview#pydrake.common.containers.namedview) 워크플로를 사용하는 것이 좋다. Drake 개발자는 [C++에서 유사한 기능](https://github.com/RobotLocomotion/drake/issues/12566)을 제공할 계획이다.

연속 상태 벡터의 메모리 구조는 이산 상태, 매개변수, 입출력 포트 데이터와 약간 다르다는 점에 주의하자. 따라서 이를 처리하는 구문은 `CopyToVector()` 및 `SetFromVector()`와 같이 약간 다른 함수를 요구한다. 자세한 내용은 이슈 [issue #9171](https://github.com/RobotLocomotion/drake/issues/9171)을 참조하자.

my_view[:]는 namedview 자체의 NumPy 배열 뷰를 가져오는 축약형으로 사용된다. 자세한 내용은 [`namedview`](https://drake.mit.edu/pydrake/pydrake.common.containers.html?highlight=namedview#pydrake.common.containers.namedview) 문서의 예제를 참조한다.

In [ ]:
# 시스템 정의하기
class NamedViewDemo(LeafSystem):
    MyDiscreteState = namedview("MyDiscreteState", ["a", "b"])
    MyContinuousState = namedview("MyContinuousState", ["x", "z", "theta"])
    MyOutput = namedview("MyOutput", ["x","a"])

    def __init__(self):
        super().__init__()

        self.DeclareDiscreteState(2)
        self.DeclarePeriodicDiscreteUpdateEvent(
            period_sec=1.0,
            offset_sec=0.0,
            update=self.DiscreteUpdate)
        self.DeclareContinuousState(3)
        self.DeclareVectorOutputPort(name="out", size=2, calc=self.CalcOutput)

    def DiscreteUpdate(self, context, discrete_values):
        discrete_state = self.MyDiscreteState(
            context.get_discrete_state_vector().value())
        continuous_state = self.MyContinuousState(
            context.get_continuous_state_vector().CopyToVector())
        next_state = self.MyDiscreteState(discrete_values.get_mutable_value())
        # Now we can compute the next state by referencing each element by name.
        next_state.a = discrete_state.a + 1
        next_state.b = discrete_state.b + continuous_state.x

    def DoCalcTimeDerivatives(self, context, derivatives):
        continuous_state = self.MyContinuousState(
            context.get_continuous_state_vector().CopyToVector())
        dstate_dt = self.MyContinuousState(continuous_state[:])
        dstate_dt.x = -continuous_state.x
        dstate_dt.z = -continuous_state.z
        dstate_dt.theta = -np.arctan2(continuous_state.z, continuous_state.x)
        derivatives.SetFromVector(dstate_dt[:])

    def CalcOutput(self, context, output):
        discrete_state = self.MyDiscreteState(
            context.get_discrete_state_vector().value())
        continuous_state = self.MyContinuousState(
            context.get_continuous_state_vector().CopyToVector())
        out = self.MyOutput(output.get_mutable_value())
        out.x = continuous_state.x
        out.a = discrete_state.a

# Instantiate the System.
system = NamedViewDemo()
simulator = Simulator(system)
context = simulator.get_mutable_context()

# Set the initial conditions.
initial_discrete_state = NamedViewDemo.MyDiscreteState([3, 4])
context.SetDiscreteState(initial_discrete_state[:])
initial_continuous_state = NamedViewDemo.MyContinuousState.Zero()
initial_continuous_state.x = 0.5
initial_continuous_state.z = 0.92
initial_continuous_state.theta = 0.23
context.SetContinuousState(initial_continuous_state[:])

# Run the simulation.
simulator.AdvanceTo(4.0)
print(
    NamedViewDemo.MyDiscreteState(context.get_discrete_state_vector().value()))
print(
    NamedViewDemo.MyContinuousState(
        context.get_continuous_state_vector().CopyToVector()))
print(NamedViewDemo.MyOutput(system.get_output_port().Eval(context)))


## Supporting Scalar Type Conversion (double, AutoDiff, and Symbolic)

시스템 프레임워크 전반에서 AutoDiff 및 Symbolic 타입을 지원하기 위해 LeafSystems는 "스칼라 유형 변환(scalar type conversion)"을 지원하도록 작성할 수 있다. Python에서는 이런 지원을 추가하려면 약간의 보일러플레이트 코드가 필요하다. 다음은 간단한 예이다.:

<table align=center cellpadding=0 cellspacing=0><tr align=center><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=right style="padding:5px 0px 5px 0px; border:none;">state &rarr;</td></tr><tr><td align=right style="padding:5px 0px 5px 0px; border:none;">command &rarr;</td></tr></table></td><td align=center style="border:2px solid black;padding-left:20px;padding-right:20px;vertical-align:middle" bgcolor=#F0F0F0>RunningCost</td><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; cost</td></tr></table></td></tr></table>

In [11]:
from pydrake.systems.framework import LeafSystem_
from pydrake.systems.scalar_conversion import TemplateSystem
from pydrake.autodiffutils import AutoDiffXd
from pydrake.symbolic import Expression

@TemplateSystem.define("RunningCost_")
def RunningCost_(T):

    class Impl(LeafSystem_[T]):

        def _construct(self, converter=None, Q=np.eye(2)):
            super().__init__(converter)
            self._Q = Q
            self._state_port = self.DeclareVectorInputPort("state", 2)
            self._command_port = self.DeclareVectorInputPort("command", 1)
            self.DeclareVectorOutputPort("cost", 1, self.CostOutput)

        def _construct_copy(self, other, converter=None):
            # Any member fields (e.g. configuration values) need to be
            # transferred here from `other` to `self`.
            Impl._construct(self, converter=converter, Q=other._Q)

        def CostOutput(self, context, output):
            x = self._state_port.Eval(context)
            u = self._command_port.Eval(context)[0]
            output[0] = x.dot(self._Q.dot(x)) + u**2

    return Impl

RunningCost = RunningCost_[None]  # Default instantiation.

중요한 단계들 :
- `@TemplateSystem` 데코레이터를 추가
- `LeafSystem` 대신 `LeafSystem_[T]`에서 상속받기
- 일반적인 `__init__` 메서드 대신 `_construct` 메서드를 구현하기
- `_construct_copy` 메서드를 구현. 이 메서드는 `_construct`와 동일한 멤버 필드를 채워야 한다(이 예에서는 self.Q로 작업).
- 기본 인스턴스화(default instantiation)를 추가. 그러면 `RunningCost_[float]`를 사용할 수 있고 해당 시스템을 계속해서 `RunningCost`로 참조할 수 있습니다.

더 자세한 내용은 C++에서의 스칼라 변환 관련 [문서](https://drake.mit.edu/doxygen_cxx/group__system__scalar__conversion.html)에서, 그리고 @TemplateSystem 데코레이터에 대한 문서는 [`@TemplateSystem`](https://drake.mit.edu/pydrake/pydrake.systems.scalar_conversion.html#pydrake.systems.scalar_conversion.TemplateSystem) decorator를 참조한다.

In [ ]:
# 원래 방식으로 system을 사용할 수 있다.:
system = RunningCost(Q=np.diag([10, 1]))
context = system.CreateDefaultContext()
system.get_input_port(0).FixValue(context, [1, 2])
system.get_input_port(1).FixValue(context, [3])
print(system.get_output_port().Eval(context))

# 시스템의 autodiff 혹은 symbolic 버전을 사용할 수도 있다.
# either by declaring them directly:
system_ad = RunningCost_[AutoDiffXd]()
system_symbolic = RunningCost_[Expression]()

# 혹은 scalar conversion:
system_ad = system.ToAutoDiffXd()
system_symbolic = system.ToSymbolic()

# time, state, parameters 그리고 필요하면 input ports를 변환할 수 있다.
# 원래 시스템으로부터 :
context_symbolic = system_symbolic.CreateDefaultContext()
context_symbolic.SetTimeStateAndParametersFrom(context)
system_symbolic.FixInputPortsFrom(system, context, context_symbolic)
print(system_symbolic.get_output_port().Eval(context_symbolic))

## 추가 자료

[Caching in the systems framework](https://drake.mit.edu/doxygen_cxx/group__cache__design__notes.html)

[Declaring events](https://drake.mit.edu/doxygen_cxx/group__events__description.html)

[Stochastic systems](https://drake.mit.edu/doxygen_cxx/group__stochastic__systems.html)

Declaring [system constraints](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_system_constraint.html): [equality](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_leaf_system.html#a4948ad0241c67045b3c794874b2986a0) and [inequality](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_leaf_system.html#a3bac306621c3f0839324649151c22af2).

[Writing continuous dynamics in implicit form](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_system.html#a2bb4c1e3572a8009863b5a342fcb5c49)